In [ ]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-large-en-v1.5", model_kwargs={"device": device}
)

In [ ]:
from dotenv import load_dotenv
from pinecone import Pinecone

load_dotenv()
pc = Pinecone()
index = pc.Index("faqs")

In [ ]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [ ]:
vector_store.similarity_search("forget password", k=3)

In [ ]:
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5},
)
retriever.invoke("how can i stay healthy?")

In [ ]:
retriever = vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 5, "lambda_mult": 0.5},
)
retriever.invoke("how can i stay healthy?")

In [ ]:
from langchain.chat_models import init_chat_model
from langchain.retrievers.multi_query import MultiQueryRetriever

question = "Forget"
llm = init_chat_model("gemini-2.5-flash", model_provider="google_genai")
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=vector_store.as_retriever(
        search_type="mmr", search_kwargs={"k": 4, "lambda_mult": 0.5}
    ),
    llm=llm,
)

In [ ]:
import torch
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

device = "cuda" if torch.cuda.is_available() else "cpu"
embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-large-en-v1.5", model_kwargs={"device": device}
)


from pinecone import Pinecone

pc = Pinecone()
index = pc.Index("faqs")


from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

from langchain.chat_models import init_chat_model
from langchain.retrievers.multi_query import MultiQueryRetriever

llm = init_chat_model("gemini-2.5-flash", model_provider="google_genai")
retriever = MultiQueryRetriever.from_llm(
    retriever=vector_store.as_retriever(
        search_type="mmr", search_kwargs={"k": 4, "lambda_mult": 0.5}
    ),
    llm=llm,
)

In [ ]:
query = "forget"

In [ ]:
retriever.invoke(query)

In [ ]:
from langchain_core.prompts import PromptTemplate

template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.

{context}

Question: {question}

Helpful Answer:"""
custom_rag_prompt = PromptTemplate.from_template(template)

In [ ]:
from langchain_core.prompts import PromptTemplate

template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.

{context}

Question: {question}

Helpful Answer:"""
custom_rag_prompt = PromptTemplate.from_template(template)

In [ ]:
from support_hub.model_hub import get_model

model = get_model()
from dotenv import load_dotenv
from pinecone import Pinecone

load_dotenv()
pc = Pinecone()
index = pc.Index("faqs")
import torch
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_pinecone import PineconeVectorStore

device = "cuda" if torch.cuda.is_available() else "cpu"
embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-large-en-v1.5", model_kwargs={"device": device}
)
vector_store = PineconeVectorStore(index=index, embedding=embeddings)
from support_hub.retriever_factory import create_multi_query_retriever

retriever = create_multi_query_retriever(vector_store)

In [ ]:
retriever.invoke("forget")

In [ ]:
custom_rag_prompt

In [ ]:
graph.invoke({"question": "How can I reset my password?"})

In [ ]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field

tagging_prompt = ChatPromptTemplate.from_template(
    """
Extract the desired information from the following passage.

Only extract the properties mentioned in the 'Classification' function.

Passage:
{input}
"""
)


class Classification(BaseModel):
    sentiment: str = Field(..., enum=["happy", "neutral", "sad"])
    aggressiveness: int = Field(
        ...,
        description="describes how aggressive the statement is, the higher the number the more aggressive",
        enum=[1, 2, 3, 4, 5],
    )
    language: str = Field(
        ..., enum=["spanish", "english", "french", "german", "italian"]
    )


# Structured LLM
structured_llm = llm.with_structured_output(Classification)

In [ ]:
"ff"

In [ ]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
class Classification(BaseModel):
    sentiment: str = Field(..., enum=["happy", "neutral", "sad"])
    aggressiveness: int = Field(
        ...,
        description="describes how aggressive the statement is, the higher the number the more aggressive",
        enum=[1, 2, 3, 4, 5],
    )
    language: str = Field(
        ..., enum=["spanish", "english", "french", "german", "italian"]
    )

In [ ]:
tagging_prompt = ChatPromptTemplate.from_template(
    """
Extract the desired information from the following passage.

Only extract the properties mentioned in the 'Classification' function.

Passage:
{input}
"""
)

a = llm.invoke(f"i forget password you are bad person\n out should be in {Classification.model_json_schema()} ")

In [ ]:
print(a.content)

In [ ]:
Classification(**{
  "sentiment": "sad",
  "aggressiveness": 5,
  "language": "english"
})

In [ ]:
from langchain.chat_models import init_chat_model
# from langchain.output_parsers import EnumOutputParser
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field, model_validator

model = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

from enum import Enum

from langchain.output_parsers import OutputFixingParser
from pydantic import BaseModel, Field, conint


class CategoriesProbabilities(BaseModel):
    Desktop: float = Field(
        ...,
        description="Probability that the ticket relates to general desktop issues, including software, settings, or performance.",
    )
    Desktop_Application: float = Field(
        ...,
        description="Probability that the ticket is about problems with installed desktop applications (e.g., MS Office, browsers).",
    )
    Desktop_Asset_Management: float = Field(
        ...,
        description="Probability that the ticket concerns desktop asset tracking, inventory, or allocation.",
    )
    Desktop_Hardware: float = Field(
        ...,
        description="Probability that the ticket is related to physical hardware issues such as monitor, keyboard, or peripherals.",
    )
    Desktop_Login_Issue: float = Field(
        ...,
        description="Probability that the ticket is about desktop login problems, like incorrect credentials or account lockouts.",
    )
    Desktop_Storage: float = Field(
        ...,
        description="Probability that the ticket involves local storage issues, such as low disk space or drive access problems.",
    )

    Email: float = Field(
        ...,
        description="Probability that the ticket relates to general email issues, including access, sync, or configuration problems.",
    )
    Email_Email_Create: float = Field(
        ...,
        description="Probability that the ticket is a request to create a new company email ID or mailbox.",
    )
    Email_Whitelist_Email: float = Field(
        ...,
        description="Probability that the ticket requests to whitelist an email address or domain to prevent blocking or spam issues.",
    )

    Internet: float = Field(
        ...,
        description="Probability that the ticket relates to internet connectivity issues, such as no access, slow speed, or frequent disconnections.",
    )
    Network: float = Field(
        ...,
        description="Probability that the ticket involves internal network problems, including LAN/Wi-Fi issues or access to shared resources.",
    )
    International_Calling: float = Field(
        ...,
        description="Probability that the ticket relates to enabling, disabling, or troubleshooting international calling services.",
    )

    Server: float = Field(
        ...,
        description="Probability that the ticket is about general server-related issues or requests not covered by subcategories.",
    )
    Server_Application: float = Field(
        ...,
        description="Probability that the ticket concerns applications hosted on servers, including downtime or access errors.",
    )
    Server_Client_Servers: float = Field(
        ...,
        description="Probability that the ticket relates to client-specific servers, configurations, or deployment problems.",
    )

    Storage: float = Field(
        ...,
        description="Probability that the ticket involves shared or cloud storage issues (e.g., OneDrive, NAS, shared drives).",
    )
    Printing: float = Field(
        ...,
        description="Probability that the ticket is about printing issues, such as printer detection, queue errors, or print quality problems.",
    )

    Website_Blocked: float = Field(
        ...,
        description="Probability that the ticket requests access to a blocked website or restricted web content.",
    )
    New_Service_Request: float = Field(
        ...,
        description="Probability that the ticket is a request for a new IT service, software installation, or hardware provisioning.",
    )

    CSG_PDG: float = Field(
        ...,
        description="Probability that the ticket is related to Corporate Services Group – Product Development Group internal tools or systems.",
    )
    Others: float = Field(
        ...,
        description="Probability that the ticket does not fall into any predefined categories and needs manual review or reassignment.",
    )

    def top_category(self) -> tuple[str, float]:
        data = self.model_dump()  # Pydantic v2
        return max(data.items(), key=lambda kv: kv[1])


class Priority(Enum):
    VeryHigh = 1
    High = 2
    Normal = 3
    Low = 4
    VeryLow = 5


class TicketPrediction(BaseModel):
    categories_probabilities: CategoriesProbabilities = Field(
        ..., description="Probability scores for each ticket category."
    )
    priority: Priority = Field(
        ...,
        description=(
            "Priority of the ticket, from 1 to 5:\n"
            "1 = Very High\n"
            "2 = High\n"
            "3 = Normal\n"
            "4 = Low\n"
            "5 = Very Low"
        ),
    )


# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=TicketPrediction)

new_parser = OutputFixingParser.from_llm(parser=parser, llm=model)

prompt = PromptTemplate(
    template="""
You are an IT support ticket classifier.

Given the ticket description, provide the following:

1. **categories_probabilities**: a probability score (in percentage) for each ticket category that represents how likely the ticket belongs to that category.
   - All probabilities must sum up to 100%.
   - No two categories should have the same probability score.

2. **priority**: one of the following Enum values (VeryHigh = 1, High = 2, Normal = 3, Low = 4, VeryLow = 5), representing how important it is to resolve the issue quickly so that there are no work blockers or wasted time.

{format_instructions}

Ticket Description:
"{query}"
""",
    input_variables=["query"],
    # partial_variables={"format_instructions": new_parser.get_format_instructions()},
    partial_variables={"format_instructions": new_parser.get_format_instructions()},
)


# And a query intended to prompt a language model to populate the data structure.
prompt_and_model = prompt | model
output = prompt_and_model.invoke({"query": "my active directory is locked"})
opt = parser.invoke(output)
print(opt.categories_probabilities.top_category())
print(opt.priority)

In [ ]:
opt.categories_probabilities.top_category()

In [ ]:
print(prompt.invoke({"query": "it is very good  my  bloody son."}).to_string())

In [ ]:
from support_hub.ticket_classifier.ticket_classification import run_ticket_classification

await run_ticket_classification("create new s3 bucket for me?")

In [ ]:
from support_hub.ticket_classifier.ticket_classification import run_ticket_classification

await run_ticket_classification("my active directory is locked")

In [ ]:
import asyncio
import aiohttp
import json

async def fetch_skills(session, base_url):
    """
    Asynchronously fetch skills data from the API endpoint.
    
    Args:
        session (aiohttp.ClientSession): The aiohttp session for making requests.
        base_url (str): The base URL of the API (e.g., 'http://example.com').
    
    Returns:
        dict: The parsed JSON response.
    """
    url = f"{base_url}/api/skills/"
    async with session.get(url) as response:
        # if response.status == 200:
            return await response.json()
        # else:
        #     raise Exception(f"Failed to fetch data: HTTP {response.status}")

async with aiohttp.ClientSession() as session:
       
            data = await fetch_skills(session, "http://192.168.71.115")
            print(data)

In [ ]:
import aiohttp

async def fetch_roles(session, base_url):
    """
    Asynchronously fetch skills data from the API endpoint.
    
    Args:
        session (aiohttp.ClientSession): The aiohttp session for making requests.
        base_url (str): The base URL of the API (e.g., 'http://example.com').
    
    Returns:
        dict: The parsed JSON response.
    """
    url = f"{base_url}/api/skills/"
    async with session.get(url) as response:
        if response.status == 200:
            return await response.json()
        else:
            raise Exception(f"Failed to fetch data: HTTP {response.status}")

async def list_roles():
    base_url = "http://192.168.71.115"  # Replace with your actual base URL
    async with aiohttp.ClientSession() as session:
        try:
            data = await fetch_roles(session, base_url)
            # Example: Access the roles
            roles = [item["role"] for item in data["data"]]
            return {"roles": roles}
        except Exception as e:
            print(f"Error: {e}")
await list_roles()

In [ ]:
roles = await list_roles()

In [ ]:
roles

In [ ]:
import aiohttp


class RoleFetcher:
    """
    A class to asynchronously fetch roles from the API.
    """

    def __init__(self, base_url: str = "http://192.168.71.115"):
        """
        Initialize the RoleFetcher with the base URL of the API.

        Args:
            base_url (str): The base URL of the API (e.g., 'http://192.168.71.115').
        """
        self.base_url = base_url

    async def fetch_roles(self):
        """
        Asynchronously fetch roles from the API endpoint.

        Returns:
            dict: A dictionary containing the list of roles.

        Raises:
            Exception: If the API request fails.
        """
        url = f"{self.base_url}/api/skills/"
        async with aiohttp.ClientSession() as session:
            async with session.get(url) as response:
                if response.status == 200:
                    data = await response.json()
                    roles = [item["role"] for item in data["data"]]
                    return {"roles": roles}
                else:
                    raise Exception(f"Failed to fetch data: HTTP {response.status}")

    async def list_roles(self):
        """
        Asynchronously list roles from the API.

        Returns:
            dict: A dictionary containing the list of roles.

        Raises:
            Exception: If the API request fails.
        """
        return await self.fetch_roles()


fetcher = RoleFetcher()

await fetcher.list_roles()

In [ ]:
from pydantic import BaseModel, Field

from support_hub.ticket_classifier.models.utils import RoleFetcher


async def get_roles():
    fetcher = RoleFetcher()
    try:
        result = await fetcher.list_roles()
        return result.get("roles", [])
    except Exception as e:
        print(f"Error: {e}")
a = await get_roles()

class Roles(BaseModel):
    role_names: list[str] = Field(default=a, description="Names of the roles")


In [ ]:
Roles(role_names=a)

In [ ]:
from typing import Literal



In [ ]:
a

In [ ]:
from support_hub.ticket_classifier.parsers.output_parsers import get_pydantic_role_parser

p = get_pydantic_role_parser()

In [ ]:
from support_hub.ticket_classifier.prompts.ticket_prompt import get_ticket_classification_prompt_roles

print(get_ticket_classification_prompt_roles(p.get_format_instructions()).invoke({"ticket": "my active directory is locked"}).to_string())

In [ ]:
from langchain import hub
from langchain_core.documents import Document
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict

from support_hub.model_hub import get_model
from support_hub.retriever_factory import create_multi_query_retriever

llm = get_model()
# prompt = hub.pull("rlm/rag-prompt")


from langchain_core.prompts import PromptTemplate

template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Keep the answer as concise as possible.

Context: {context}

Question: {question}

Answer:"""
prompt = PromptTemplate.from_template(template)


import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

from langchain_huggingface.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-large-en-v1.5", model_kwargs={"device": device}
)
from dotenv import load_dotenv
from pinecone import Pinecone

load_dotenv()
pc = Pinecone()
index = pc.Index("faqs")
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)


# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str
    answer_found: bool


# Define application steps
def retrieve(state: State):
    # retrieved_docs = vector_store.similarity_search(state["question"])
    retrieved_docs = create_multi_query_retriever(vector_store=vector_store).invoke(
        state["question"]
    )
    return {"context": retrieved_docs}


from support_hub.context_evaluateor import evaluate_relevance


async def generate(state: State):
    docs_content = (
        "\n```\n" + "\n\n".join(doc.page_content for doc in state["context"]) + "\n```"
    )
    # print(docs_content)

    evaluated_class = await evaluate_relevance(
        query=state["question"], retrieved_context=docs_content
    )
    # print("end=  ", evaluated_class.get("reason"))
    if evaluated_class.get("is_relevant") is False:
        return {"answer_found": False}

    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer_found": True, "answer": response.content}


# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()
answer = await graph.ainvoke({"question": "How can I reset my password??"})
print(answer.get("answer_found"))
if answer.get("answer_found"):
    print(answer["answer"])

In [ ]:
"""Following are the retrieved context. give answer according to these context only"""

In [ ]:
answer = graph.invoke({"question": "How can I reset my password?"})

In [ ]:
answer["context"]

In [ ]:
answer["answer"]

In [ ]:
from typing import List, Optional

import torch
from dotenv import load_dotenv
from langchain_core.documents import Document
from langchain_core.language_models.base import BaseLanguageModel
from langchain_core.prompts import PromptTemplate
from langchain_core.vectorstores import VectorStore
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_pinecone import PineconeVectorStore
from langgraph.graph import START, StateGraph
from pinecone import Pinecone
from typing_extensions import TypedDict

from support_hub.context_evaluateor import evaluate_relevance
from support_hub.model_hub import get_model
from support_hub.retriever_factory import create_multi_query_retriever


# State definition
class RAGState(TypedDict):
    """State schema for RAG pipeline."""

    question: str
    context: List[Document]
    answer: str
    answer_found: bool


class RAGConfig:
    """Configuration for RAG pipeline components."""

    # Prompt template
    ANSWER_TEMPLATE = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Keep the answer as concise as possible.

Context: {context}

Question: {question}

Answer:"""

    # Embedding model configuration
    EMBEDDING_MODEL_NAME = "BAAI/bge-large-en-v1.5"

    # Pinecone configuration
    PINECONE_INDEX_NAME = "faqs"

    def __init__(
        self,
        embedding_model_name: Optional[str] = None,
        pinecone_index_name: Optional[str] = None,
        answer_template: Optional[str] = None,
    ):
        """
        Initialize RAG configuration.

        Args:
            embedding_model_name: Name of HuggingFace embedding model.
            pinecone_index_name: Name of Pinecone index.
            answer_template: Custom prompt template for answer generation.
        """
        self.embedding_model_name = embedding_model_name or self.EMBEDDING_MODEL_NAME
        self.pinecone_index_name = pinecone_index_name or self.PINECONE_INDEX_NAME
        self.answer_template = answer_template or self.ANSWER_TEMPLATE


class RAGPipeline:
    """End-to-end RAG pipeline using LangGraph."""

    def __init__(
        self,
        vector_store: Optional[VectorStore] = None,
        llm: Optional[BaseLanguageModel] = None,
        config: Optional[RAGConfig] = None,
    ):
        """
        Initialize RAG pipeline.

        Args:
            vector_store: Vector store for document retrieval. If None, creates Pinecone store.
            llm: Language model for generation. If None, uses default from model_hub.
            config: RAG configuration. If None, uses default configuration.
        """
        self.config = config or RAGConfig()
        self.llm = llm or get_model()
        self.vector_store = vector_store or self._create_default_vector_store()
        self.prompt = PromptTemplate.from_template(self.config.answer_template)
        self.graph = self._build_graph()

    def _create_default_vector_store(self) -> VectorStore:
        """Create default Pinecone vector store with HuggingFace embeddings."""
        load_dotenv()

        # Set device for embeddings
        device = "cuda" if torch.cuda.is_available() else "cpu"

        # Initialize embeddings
        embeddings = HuggingFaceEmbeddings(
            model_name=self.config.embedding_model_name, model_kwargs={"device": device}
        )

        # Initialize Pinecone
        pc = Pinecone()
        index = pc.Index(self.config.pinecone_index_name)

        return PineconeVectorStore(index=index, embedding=embeddings)

    def _retrieve(self, state: RAGState) -> dict:
        """
        Retrieve relevant documents for the question.

        Args:
            state: Current RAG state.

        Returns:
            Dictionary with retrieved context documents.
        """
        retriever = create_multi_query_retriever(vector_store=self.vector_store)
        retrieved_docs = retriever.invoke(state["question"])
        return {"context": retrieved_docs}

    async def _generate(self, state: RAGState) -> dict:
        """
        Generate answer from retrieved context.

        Args:
            state: Current RAG state with retrieved context.

        Returns:
            Dictionary with answer_found flag and answer content.
        """
        # Format context documents
        docs_content = (
            "\n```"
            + "\n\n".join(doc.page_content for doc in state["context"])
            + "\n```"
        )

        # Evaluate context relevance
        evaluated_class = await evaluate_relevance(
            query=state["question"], retrieved_context=docs_content
        )

        # Return early if context is not relevant
        if not evaluated_class.get("is_relevant"):
            return {"answer_found": False}

        # Generate answer
        messages = self.prompt.invoke(
            {"question": state["question"], "context": docs_content}
        )
        response = self.llm.invoke(messages)

        return {"answer_found": True, "answer": response.content}

    def _build_graph(self) -> StateGraph:
        """
        Build LangGraph pipeline.

        Returns:
            Compiled StateGraph for RAG pipeline.
        """
        graph_builder = StateGraph(RAGState).add_sequence(
            [self._retrieve, self._generate]
        )
        graph_builder.add_edge(START, "_retrieve")
        return graph_builder.compile()

    async def ainvoke(self, question: str) -> dict:
        """
        Asynchronously invoke RAG pipeline.

        Args:
            question: User question to answer.

        Returns:
            Dictionary containing answer_found flag and answer content.
        """
        result = await self.graph.ainvoke({"question": question})
        return {
            "answer_found": result.get("answer_found", False),
            "answer": result.get("answer"),
        }

    def invoke(self, question: str) -> dict:
        """
        Synchronously invoke RAG pipeline.

        Args:
            question: User question to answer.

        Returns:
            Dictionary containing answer_found flag and answer content.
        """
        result = self.graph.invoke({"question": question})
        return {
            "answer_found": result.get("answer_found", False),
            "answer": result.get("answer"),
        }

    async def stream(self, question: str):
        """
        Stream RAG pipeline execution with intermediate states.

        Args:
            question: User question to answer.

        Yields:
            Intermediate states during pipeline execution.
        """
        async for state in self.graph.astream({"question": question}):
            yield state


# Factory function for easy instantiation
async def create_rag_pipeline(
    ticket: str,
    vector_store: Optional[VectorStore] = None,
    llm: Optional[BaseLanguageModel] = None,
    config: Optional[RAGConfig] = None,
) -> RAGPipeline:
    """
    Create a RAG pipeline instance.

    Args:
        vector_store: Custom vector store. If None, uses default Pinecone.
        llm: Custom language model. If None, uses default from model_hub.
        config: Custom RAG configuration. If None, uses defaults.

    Returns:
        Configured RAGPipeline instance.
    """
    pipeline = RAGPipeline(vector_store=vector_store, llm=llm, config=config)
    result = await pipeline.ainvoke(ticket)
    return {"answer_found": result.get("answer_found"), "answer": result.get("answer")}


# Create pipeline with defaults
pipeline = await create_rag_pipeline("my active directory is locked")

# # Get answer
# result = await pipeline.ainvoke("hahaha")

# if result["answer_found"]:
#     print(f"Answer: {result['answer']}")
# else:
#     print("No relevant answer found in knowledge base.")
pipeline

In [1]:
from support_hub.rag_pipeline import create_rag_pipeline
print(create_rag_pipeline("my active directory is locked"))

/mnt/d/Sourabh_critical/ids-hackathon/backend/ai/.venv/lib/python3.12/site-packages/langchain_pinecone/__init__.py:3: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_pinecone.vectorstores import Pinecone, PineconeVectorStore
E0000 00:00:1763109632.863861  235497 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.
E0000 00:00:1763109634.378801  235497 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is no

{'answer_found': False, 'answer': None}


In [4]:
create_rag_pipeline("i forget my password")

E0000 00:00:1763109046.195574  232672 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.
E0000 00:00:1763109046.219038  232672 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.
E0000 00:00:1763109053.736055  232672 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


{'answer_found': False, 'answer': None}

In [4]:
# print(create_rag_pipeline("I want to configure FortiClient VPN gateway settings").get("answer"))
print(create_rag_pipeline("""FortiClient VPN Configuration Request\n\nHi IT Team,

I want to configure FortiClient VPN gateway settings"

Thanks,
Davinder singh
sr. software Engineer

""").get("answer"))

E0000 00:00:1763111763.224533  235497 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.
E0000 00:00:1763111763.243450  235497 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.
E0000 00:00:1763111778.645577  235497 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


To configure your FortiClient VPN gateway settings, please follow these steps:

1.  Open FortiClient and navigate to the VPN settings.
2.  Click 'Add a new connection' to create a new VPN gateway.
3.  Enter the connection name, description, and remote gateway details as required.
4.  Save the settings and attempt to connect to the VPN.

**Important Note:** If the VPN does not connect, please try removing the gateways one by one and testing the connection after each removal. Once you successfully connect, you can then add the gateways back again.
